## Fuzzy Matching School Names

Matches school name in educator data with their ids.

In [162]:
import os
import pandas as pd
from fuzzywuzzy import fuzz, process
from collections import defaultdict

In [163]:
input_data_dir = "data-to-match"

educators_inexperienced_path = os.path.join(input_data_dir, "Educators_Inexperienced_2018_JAN_24th_2019.csv")
educators_out_of_field_path = os.path.join(input_data_dir, "Educators_OUT_OF_FIELD_2018_JAN_24th_2019.csv")
school_id_data_path = os.path.join(input_data_dir, "ga_public_school_contact_list_Ethan.xlsx")

educators_inexperienced = pd.read_csv(educators_inexperienced_path)
educators_out_of_field = pd.read_csv(educators_out_of_field_path)
school_id_data = pd.read_excel(school_id_data_path, sheet_name="Sheet_1")


In [164]:
# Cleaning Public School Data.
school_id_data["SCHOOL_NAME"] = school_id_data["SCHOOL_NAME"].str.upper()
school_id_data["SCHOOL_NAME"] = school_id_data["SCHOOL_NAME"].str.strip()
school_id_data["SYSTEM_NAME"] = school_id_data["SYSTEM_NAME"].str.upper()
school_id_data["SYSTEM_NAME"] = school_id_data["SYSTEM_NAME"].str.strip()
# Must take care of duplicates separately. Some schools have the same name but are differet schools with different IDs.
school_id_data_no_duplicates = school_id_data.drop_duplicates(subset="SCHOOL_NAME", keep=False)

In [165]:
# Cleaning Educators Inexperienced.
educators_inexperienced["INSTN_NAME"] = educators_inexperienced["INSTN_NAME"].str.upper()
educators_inexperienced["INSTN_NAME"] = educators_inexperienced["INSTN_NAME"].str.strip()
educators_inexperienced["SCHOOL_DSTRCT_NM"] = educators_inexperienced["SCHOOL_DSTRCT_NM"].str.upper()
educators_inexperienced["SCHOOL_DSTRCT_NM"] = educators_inexperienced["SCHOOL_DSTRCT_NM"].str.strip()
educators_inexperienced = educators_inexperienced[~educators_inexperienced["INSTN_NAME"].str.endswith("ALL SCHOOLS")]

In [166]:
# Cleaning Educators Out of Field.
educators_out_of_field["INSTN_NAME"] = educators_out_of_field["INSTN_NAME"].str.upper()
educators_out_of_field["INSTN_NAME"] = educators_out_of_field["INSTN_NAME"].str.strip()
educators_out_of_field["SCHOOL_DSTRCT_NM"] = educators_out_of_field["SCHOOL_DSTRCT_NM"].str.upper()
educators_out_of_field["SCHOOL_DSTRCT_NM"] = educators_out_of_field["SCHOOL_DSTRCT_NM"].str.strip()
educators_out_of_field = educators_out_of_field[~educators_out_of_field["INSTN_NAME"].str.endswith("ALL SCHOOLS")]

### Matching for Inexperienced Educators

In [167]:
inexperienced_with_perfect_match = educators_inexperienced[educators_inexperienced["INSTN_NAME"].isin(school_id_data["SCHOOL_NAME"])]
inexperienced_no_perfect_match = educators_inexperienced[~educators_inexperienced["INSTN_NAME"].isin(school_id_data["SCHOOL_NAME"])]

print("Shape of inexperienced_with_perfect_match: {}".format(inexperienced_with_perfect_match.shape))
print("Shape of inexperienced_no_perfect_match: {}".format(inexperienced_no_perfect_match.shape))

Shape of inexperienced_with_perfect_match: (5587, 8)
Shape of inexperienced_no_perfect_match: (97, 8)


In [168]:
print("Number of schools in inexperienced data: {}".format(len(educators_inexperienced)))
print("Number of schools in inexperienced data with perfect match: {}".format(len(inexperienced_with_perfect_match)))
print("Number of schools in inexperienced data with no perfect match: {}".format(len(inexperienced_no_perfect_match)))

correct_size = inexperienced_with_perfect_match.shape[0] + inexperienced_no_perfect_match.shape[0] == educators_inexperienced.shape[0]
print("Sizes of perfect and non-perfect subsets is correct: {}".format(correct_size))

no_overlap = len(set(inexperienced_with_perfect_match["INSTN_NAME"]).intersection(set(inexperienced_no_perfect_match["INSTN_NAME"]))) == 0
print("No overlap between perfect and non-perfect matching subsets: {}".format(no_overlap))

Number of schools in inexperienced data: 5684
Number of schools in inexperienced data with perfect match: 5587
Number of schools in inexperienced data with no perfect match: 97
Sizes of perfect and non-perfect subsets is correct: True
No overlap between perfect and non-perfect matching subsets: True


In [170]:
merged_inexperienced = inexperienced_with_perfect_match.merge(
    school_id_data,
    left_on=["INSTN_NAME", "SCHOOL_DSTRCT_NM"],
    right_on=["SCHOOL_NAME", "SYSTEM_NAME"],
    how="left",
)

merged_inexperienced["MATCH_TYPE"] = "PERFECT"

print("Shape of merged_inexperienced: {}".format(merged_inexperienced.shape))

Shape of merged_inexperienced: (5587, 30)


In [173]:
unmatched_by_county = merged_inexperienced[merged_inexperienced["Full School_ID"].isnull()]
print("Number of unmatched schools by county: {}".format(len(unmatched_by_county)))



Number of unmatched schools by county: 272


In [121]:
def fuzzy_match_school_name(name):
    """Matches school name to school id using fuzzy matching.
    
    Parameters
    ----------
    name : str
        School name to match.
        
    Returns
    -------
    name : str
        Tuple of matched name, match confidence score, and school id.
    """
    match = process.extractOne(name, school_id_data_no_duplicates["SCHOOL_NAME"])
    name = match[0]
    score = match[1]
    row = match[2]
    school_id = school_id_data_no_duplicates.loc[row]["Full School_ID"]
    return (name, score, school_id)

In [122]:
inexperienced_no_perfect_match["SCHOOL_ID"] = inexperienced_no_perfect_match["INSTN_NAME"].apply(fuzzy_match_school_name)
inexperienced_no_perfect_match["MATCH_TYPE"] = "FUZZY"

C:\Users\ethan_haque\AppData\Local\Temp\ipykernel_47136\604840341.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inexperienced_no_perfect_match["SCHOOL_ID"] = inexperienced_no_perfect_match["INSTN_NAME"].apply(fuzzy_match_school_name)
C:\Users\ethan_haque\AppData\Local\Temp\ipykernel_47136\604840341.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inexperienced_no_perfect_match["MATCH_TYPE"] = "FUZZY"


### Matching for Out of Field Educators

### Saving Output

In [123]:
output_dir = "matched-data"

In [124]:
# rememeber to remove non-alphanumeric characters from school names.

inexperienced_matched = pd.concat([inexperienced_with_perfect_match, inexperienced_no_perfect_match])
inexperienced_matched.to_csv(os.path.join(output_dir, "inexperienced-matched.csv"), index=False)
# Cleaned by hand.

# merged_inexperienced[
#     [
#         "LONG_SCHOOL_YEAR",
#         "SCHOOL_DSTRCT_NM",
#         "SYSTEM_NAME",
#         "INSTN_NAME",
#         "SCHOOL_NAME",
#         "LABEL_LVL_3_DESC",
#         "LABEL_LVL_2_DESC",
#         "FTE",
#         "INEXPERIENCED_FTE",
#         "INEXPERIENCED_FTE_PCT",
#         "Full School_ID",
#     ]
# ].to_csv("inexperienced_with_perfect_match.csv", index=False)
